# Training & Evaluation  

## Importing Preprocessing Pipeline

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline, make_pipeline

In [2]:
preprocessing_pipeline = None  # Define as a placeholder

%run "02_transform_data.ipynb"

In [3]:
preprocessing_pipeline

ColumnTransformer(transformers=[('bedrooms_per_room',
                                 Pipeline(steps=[('impute_ratios',
                                                  SimpleImputer(strategy='median')),
                                                 ('calculate_ratios',
                                                  FunctionTransformer(feature_names_out=<function calculate_ratio_feature_names at 0x7f2cac7882c0>,
                                                                      func=<function calculate_ratio at 0x7f2cac788720>)),
                                                 ('standard_scaler',
                                                  StandardScaler())]),
                                 ['total_bedrooms', 'total_roo...
                                                                      func=<function heavy_tail_distribution at 0x7f2cac7ad080>,
                                                                      inverse_func=<ufunc 'expm1'>)),
                                                 ('standard_scaler',
                                                  StandardScaler())]),
                                 ['total_bedrooms', 'total_rooms', 'population',
                                  'households', 'median_income']),
                                ('categorical',
                                 Pipeline(steps=[('impute_categories',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encode_categories',
                                                  OneHotEncoder(sparse_output=False))]),
                                 ['ocean_proximity'])])

## Load Training Data

In [4]:
processed_data_path = Path("..", "data", "processed", "housing")

In [5]:
## read data
data = pd.read_csv(Path(processed_data_path, "train_set.csv"))

## Split Features & Labels

In [6]:
## before we create the pipeline lets split he training data into features and labels
df_features = data.drop("median_house_value", axis=1)
df_labels = data["median_house_value"].copy()

In [7]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16512 entries, 0 to 16511
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   longitude              16512 non-null  float64
 1   latitude               16512 non-null  float64
 2   housing_median_age     16512 non-null  float64
 3   total_rooms            16512 non-null  float64
 4   total_bedrooms         16344 non-null  float64
 5   population             16512 non-null  float64
 6   households             16512 non-null  float64
 7   median_income          16512 non-null  float64
 8   ocean_proximity        16512 non-null  object 
 9   income_categories      16512 non-null  int64  
 10  population_categories  16512 non-null  int64  
dtypes: float64(8), int64(2), object(1)
memory usage: 1.4+ MB


In [8]:
df_features.sample(10)


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,income_categories,population_categories
8670,-118.26,33.94,45.0,1080.0,218.0,850.0,237.0,2.2500,<1H OCEAN,2,1
10,-121.54,39.08,23.0,1076.0,216.0,724.0,197.0,2.3598,INLAND,2,1
16241,-121.01,37.33,17.0,1926.0,410.0,1054.0,321.0,1.6214,INLAND,2,1
12916,-122.29,37.81,46.0,935.0,297.0,582.0,277.0,0.7286,NEAR BAY,1,1
5419,-117.31,34.07,40.0,2936.0,732.0,2024.0,676.0,2.1139,INLAND,2,1
8676,-122.40,38.53,24.0,1741.0,289.0,564.0,231.0,3.6118,INLAND,3,1
14688,-117.17,34.28,13.0,4867.0,718.0,780.0,250.0,7.1997,INLAND,5,1
15440,-121.37,38.63,37.0,494.0,86.0,253.0,99.0,4.8194,INLAND,4,1
1100,-123.00,38.33,8.0,3223.0,637.0,851.0,418.0,5.6445,NEAR OCEAN,4,1
8596,-121.46,38.54,36.0,1825.0,411.0,1226.0,391.0,1.5292,INLAND,2,1


## Preprocessing Data

In [9]:
preprocessed_data = pd.DataFrame(preprocessing_pipeline.fit_transform(df_features), columns=preprocessing_pipeline.get_feature_names_out())

In [10]:
preprocessed_data.shape

(16512, 27)

In [11]:
## sanity check to see if we have empty values
preprocessed_data.isna().sum()

bedrooms_per_room__ratio                        0
rooms_per_household__ratio                      0
population_per_household__ratio                 0
multimodal_similarity__similarity_to_peak_17    0
multimodal_similarity__similarity_to_peak_26    0
multimodal_similarity__similarity_to_peak_35    0
multimodal_similarity__similarity_to_peak_52    0
cluster_similarity__similarity_to_cluster_0     0
cluster_similarity__similarity_to_cluster_1     0
cluster_similarity__similarity_to_cluster_2     0
cluster_similarity__similarity_to_cluster_3     0
cluster_similarity__similarity_to_cluster_4     0
cluster_similarity__similarity_to_cluster_5     0
cluster_similarity__similarity_to_cluster_6     0
cluster_similarity__similarity_to_cluster_7     0
cluster_similarity__similarity_to_cluster_8     0
cluster_similarity__similarity_to_cluster_9     0
log_pipeline__total_bedrooms                    0
log_pipeline__total_rooms                       0
log_pipeline__population                        0


## Training Linear Regression

### Training

In [12]:
from sklearn.linear_model import LinearRegression

linear_reg = make_pipeline(preprocessing_pipeline, LinearRegression())
linear_reg.fit(df_features, df_labels)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('bedrooms_per_room',
                                                  Pipeline(steps=[('impute_ratios',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('calculate_ratios',
                                                                   FunctionTransformer(feature_names_out=<function calculate_ratio_feature_names at 0x7f2cac7882c0>,
                                                                                       func=<function calculate_ratio at 0x7f2cac788720>)),
                                                                  ('standard_scaler',
                                                                   StandardSca...
                                                                                       inverse_func=<ufunc 'expm1'>)),
                                                                  ('standard_scaler',
                                                                   StandardScaler())]),
                                                  ['total_bedrooms',
                                                   'total_rooms', 'population',
                                                   'households',
                                                   'median_income']),
                                                 ('categorical',
                                                  Pipeline(steps=[('impute_categories',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode_categories',
                                                                   OneHotEncoder(sparse_output=False))]),
                                                  ['ocean_proximity'])])),
                ('linearregression', LinearRegression())])

### Predictions

In [13]:
predictions = linear_reg.predict(df_features)
predictions[:5].round(2)

array([263335.29, 373549.57, 112870.95,  92243.99, 316741.99])

### Evaluations

#### Calculating Error Ratios

In [14]:
## lets compute error ratios
error_ratios = predictions[:5].round(-2) / df_labels.iloc[:5].values - 1
print(", ".join([f"{100 * ratio:.1f}%" for ratio in error_ratios]))

-42.5%, -22.8%, 11.0%, -4.1%, -12.5%


#### Calculating RMSE

In [15]:
from sklearn.metrics import root_mean_squared_error

rmse = root_mean_squared_error(df_labels, predictions)
rmse

66550.34931224312

Interpretation:
* On average the value predicted by our model has difference of `65550` then the actual value. 
* Based on project requirements since the output is fed to another ML model to predict whether we should invest in this area or not, a diff of `66K` might not give us any useful indicator. 
* Also since we are testing on training data, this difference indicates that we are underfitting the model. 


#### Calculating Relative RMSE

In [16]:
mean_price = df_labels.mean()
rmse_relative = rmse / mean_price
print(f"Relative RMSE: {rmse_relative:.2%}")


Relative RMSE: 32.25%


Interpretation
* Based on project requirement the current process is costly and time-consuming and estimates are off by more than 30%, our model is not performing better than the current process.
* We need to test other models, but before that lets run cross-validation to confirm our findings

#### Cross Validation

In [17]:
from sklearn.model_selection import cross_val_score

scores = -cross_val_score(linear_reg, df_features, df_labels, scoring="neg_root_mean_squared_error", cv=10)
scores

array([66975.41917719, 66419.43069432, 64127.57598067, 75625.56219204,
       66741.20322301, 66746.47979067, 65562.44383051, 68892.13827255,
       66147.29254267, 67764.54064592])

Interpretation:
* Looks like our findings are correct, we are underfitting and getting similar RMSE values for all models
* Which means either our model is too simple or we don't have sufficient features. We need to try different models

## Random Forest

### Training

In [18]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = make_pipeline(preprocessing_pipeline, RandomForestRegressor(n_estimators=100, random_state=42))
forest_reg.fit(df_features, df_labels)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('bedrooms_per_room',
                                                  Pipeline(steps=[('impute_ratios',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('calculate_ratios',
                                                                   FunctionTransformer(feature_names_out=<function calculate_ratio_feature_names at 0x7f2cac7882c0>,
                                                                                       func=<function calculate_ratio at 0x7f2cac788720>)),
                                                                  ('standard_scaler',
                                                                   StandardSca...
                                                                  ('standard_scaler',
                                                                   StandardScaler())]),
                                                  ['total_bedrooms',
                                                   'total_rooms', 'population',
                                                   'households',
                                                   'median_income']),
                                                 ('categorical',
                                                  Pipeline(steps=[('impute_categories',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode_categories',
                                                                   OneHotEncoder(sparse_output=False))]),
                                                  ['ocean_proximity'])])),
                ('randomforestregressor',
                 RandomForestRegressor(random_state=42))])

### Predictions

In [19]:
predictions = forest_reg.predict(df_features)

In [20]:
predictions[:5].round(2)

array([433104.15, 478151.22, 106536.  , 101683.  , 370400.12])

### Evaluations

#### Calculating RMSE

In [21]:
rmse = root_mean_squared_error(df_labels, predictions)
rmse

17557.540624138303

Interpretations:
* RMSE is significanly less that `Linear Regression`
* One reason could be overfitting, we can confirm that using cross validation.
* If we are not overfitting, then we can use `GridSearch` to find the best params. 

#### Calcuating Relative RMSE

In [22]:
## calculate relative rmse
rmse_relative = rmse / mean_price
print(f"Relative RMSE: {rmse_relative:.2%}")

Relative RMSE: 8.51%


Interpretation:
* Seems like this model performs a lot better than our previous model, and than manual process. 
* If this model is not overfitting then it could be a candidate for production

#### Cross Validation

In [23]:
## cross validation
scores = -cross_val_score(forest_reg, df_features, df_labels, scoring="neg_root_mean_squared_error", cv=10)
scores

array([46518.21599266, 47657.09029417, 46009.61776303, 47236.10087738,
       46343.77184697, 47318.70351497, 47534.58520817, 49669.7806786 ,
       47584.23865129, 46877.0007688 ])

In [24]:
scores.mean(), scores.std()

(47274.91055960402, 961.6083961071712)

Interpretations:
* We are getting different numbers, which are slightly better than `LinearRegression` but a lot worse than the model trained on whole training dataset. 
* Which means we are overfitting a bit when we train using the complete dataset. 

## Support Vector Regressor

### Training

In [25]:
## using support vector regression
from sklearn.svm import SVR

svm_reg = make_pipeline(preprocessing_pipeline, SVR(kernel="rbf"))
svm_reg.fit(df_features, df_labels)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('bedrooms_per_room',
                                                  Pipeline(steps=[('impute_ratios',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('calculate_ratios',
                                                                   FunctionTransformer(feature_names_out=<function calculate_ratio_feature_names at 0x7f2cac7882c0>,
                                                                                       func=<function calculate_ratio at 0x7f2cac788720>)),
                                                                  ('standard_scaler',
                                                                   StandardSca...
                                                                                       func=<function heavy_tail_distribution at 0x7f2cac7ad080>,
                                                                                       inverse_func=<ufunc 'expm1'>)),
                                                                  ('standard_scaler',
                                                                   StandardScaler())]),
                                                  ['total_bedrooms',
                                                   'total_rooms', 'population',
                                                   'households',
                                                   'median_income']),
                                                 ('categorical',
                                                  Pipeline(steps=[('impute_categories',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode_categories',
                                                                   OneHotEncoder(sparse_output=False))]),
                                                  ['ocean_proximity'])])),
                ('svr', SVR())])

### Predictions

In [26]:
predictions = svm_reg.predict(df_features)
predictions[:5].round(2)

array([179127.79, 179704.09, 178648.92, 178747.33, 179149.84])

Interpretations:
* This is worse than both the models, may be I need to pass separate hyperparameters. 
* May be we are underfitting, lets calculate evaluation metrics and then run cross validation to be sure. 

### Evaluations

#### Calculating RMSE

In [27]:
rmse = root_mean_squared_error(df_labels, predictions)
rmse

118239.52161044904

#### Calculating Relative RMSE

In [28]:
## relative rmse
rmse_relative = rmse / mean_price
print(f"Relative RMSE: {rmse_relative:.2%}")

Relative RMSE: 57.31%


#### Cross Validation

In [29]:
## cross validation
scores = -cross_val_score(svm_reg, df_features, df_labels, scoring="neg_root_mean_squared_error", cv=10)
scores

array([120152.86799787, 121572.35148253, 115886.31068152, 117475.34228705,
       117464.73330846, 119809.06945733, 118817.52228608, 115841.63837712,
       115006.83130508, 120468.94123383])

Interpretation:
* Similar result, this might not be the right model for our data.  Or we might need hyperparameter tuning. 
* I think for now, lets focus on Random Forest hyper parameter tuning and see if we can find the right params. 

## Hyper Parameter Tuning

### Random Forest

* We can use the following params for hyper param tuning
    * n_estimators (Random Forest Regressor)
    * max_features (Random Forest Regressor)
    * n_clusters (ClusterSimilarityTransformer)
    * gamma (ClusterSimilarityTransformer)

#### Grid Search

In [30]:
# hyper parameter tuning for random forest
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

random_forest_pipeline = Pipeline(
    [("preprocessing", preprocessing_pipeline), ("randomforestregressor", RandomForestRegressor())])

param_grid = [
    {"randomforestregressor__n_estimators": [3, 10, 30], 
     "randomforestregressor__max_features": [2, 4, 6, 8], 
     "randomforestregressor__bootstrap": [False], 
     "preprocessing__cluster_similarity__calculate_cluster__n_clusters": [2, 3, 4, 10],
     "preprocessing__cluster_similarity__calculate_cluster__gamma": [0.1, 0.5, 1.0, 2.0]}
]

grid_search = GridSearchCV(random_forest_pipeline, param_grid, cv=5,
                           scoring="neg_mean_squared_error", return_train_score=True)
grid_search.fit(df_features, df_labels)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('bedrooms_per_room',
                                                                         Pipeline(steps=[('impute_ratios',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('calculate_ratios',
                                                                                          FunctionTransformer(feature_names_out=<function calculate_ratio_feature_names at 0x7f2cac7882c0>,
                                                                                                              func=<function calculate_ratio at 0x7f2cac788720>)),
                                                                                         ('stan...
             param_grid=[{'preprocessing__cluster_similarity__calculate_cluster__gamma': [0.1,
                                                                                          0.5,
                                                                                          1.0,
                                                                                          2.0],
                          'preprocessing__cluster_similarity__calculate_cluster__n_clusters': [2,
                                                                                               3,
                                                                                               4,
                                                                                               10],
                          'randomforestregressor__bootstrap': [False],
                          'randomforestregressor__max_features': [2, 4, 6, 8],
                          'randomforestregressor__n_estimators': [3, 10, 30]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [31]:
## best hyper parameters
grid_search.best_params_

{'preprocessing__cluster_similarity__calculate_cluster__gamma': 0.1,
 'preprocessing__cluster_similarity__calculate_cluster__n_clusters': 10,
 'randomforestregressor__bootstrap': False,
 'randomforestregressor__max_features': 4,
 'randomforestregressor__n_estimators': 30}

In [32]:
## best estimator
grid_search.best_estimator_

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('bedrooms_per_room',
                                                  Pipeline(steps=[('impute_ratios',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('calculate_ratios',
                                                                   FunctionTransformer(feature_names_out=<function calculate_ratio_feature_names at 0x7f2cac7882c0>,
                                                                                       func=<function calculate_ratio at 0x7f2cac788720>)),
                                                                  ('standard_scaler',
                                                                   StandardScaler(...
                                                                   StandardScaler())]),
                                                  ['total_bedrooms',
                                                   'total_rooms', 'population',
                                                   'households',
                                                   'median_income']),
                                                 ('categorical',
                                                  Pipeline(steps=[('impute_categories',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode_categories',
                                                                   OneHotEncoder(sparse_output=False))]),
                                                  ['ocean_proximity'])])),
                ('randomforestregressor',
                 RandomForestRegressor(bootstrap=False, max_features=4,
                                       n_estimators=30))])

In [33]:
## function to convert params to dataframe
def summarize_search_resutls(mean_scores, params):
    ## scrore array
    grid_search_score = []

    for mean_score, params in zip(mean_scores, params):
        ## convert params to a dictionary
        params_dict = {}
        for key, value in params.items():
            params_dict[key] = value
        grid_search_score.append({"rmse": np.sqrt(-mean_score), **params_dict})  # store the results in a list

    return pd.DataFrame(grid_search_score)

In [34]:
## create a dataframe of the results for better analysis
## evaluation scores
cvres = grid_search.cv_results_    
grid_search_score_df = summarize_search_resutls(cvres["mean_test_score"], cvres["params"])
grid_search_score_df.sort_values("rmse", ascending=True).head()

,rmse,preprocessing__cluster_similarity__calculate_cluster__gamma,preprocessing__cluster_similarity__calculate_cluster__n_clusters,randomforestregressor__bootstrap,randomforestregressor__max_features,randomforestregressor__n_estimators
41,43696.227686,0.1,10,False,4,30
44,43994.399765,0.1,10,False,6,30
92,44163.007889,0.5,10,False,6,30
89,44396.577758,0.5,10,False,4,30
140,44397.014299,1.0,10,False,6,30


Interpretation:
* So the best mean score is similar to what we had seen earlier. 
* Lets try randomized search to see if we get better params.

#### Randomized Search

In [35]:
# randomized search for hyper parameter tuning
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
    "randomforestregressor__n_estimators": randint(low=1, high=200),
    "randomforestregressor__max_features": randint(low=1, high=8),
    "preprocessing__cluster_similarity__calculate_cluster__n_clusters": randint(low=10, high=100),
}

rnd_search = RandomizedSearchCV(random_forest_pipeline, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring="neg_mean_squared_error", random_state=42)
rnd_search.fit(df_features, df_labels)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(transformers=[('bedrooms_per_room',
                                                                               Pipeline(steps=[('impute_ratios',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('calculate_ratios',
                                                                                                FunctionTransformer(feature_names_out=<function calculate_ratio_feature_names at 0x7f2cac7882c0>,
                                                                                                                    func=<function calculate_ratio at 0x7f2cac788720>)),...
                   param_distributions={'preprocessing__cluster_similarity__calculate_cluster__n_clusters': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f2ca81b89e0>,
                                        'randomforestregressor__max_features': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f2cac2b9340>,
                                        'randomforestregressor__n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f2cac2b9370>},
                   random_state=42, scoring='neg_mean_squared_error')

In [36]:
rnd_search.best_params_

{'preprocessing__cluster_similarity__calculate_cluster__n_clusters': 39,
 'randomforestregressor__max_features': 6,
 'randomforestregressor__n_estimators': 130}

In [37]:
cvres = rnd_search.cv_results_
grid_search_score_df = summarize_search_resutls(cvres["mean_test_score"], cvres["params"])
grid_search_score_df.sort_values("rmse", ascending=True).head()

,rmse,preprocessing__cluster_similarity__calculate_cluster__n_clusters,randomforestregressor__max_features,randomforestregressor__n_estimators
6,41675.589504,39,6,130
2,42009.961352,92,7,75
9,42054.953743,31,5,89
8,42381.333851,42,4,58
3,42520.090349,84,5,100


Interpretation:
* Interesting this is slightly better than before, not a huge difference.
* Lets see if we can reduce the number of columns and just focus on important ones for training. 

In [38]:
## mapping important feature names to their importance scores
feature_importances = rnd_search.best_estimator_.steps[1][1].feature_importances_
important_feature_list = pd.DataFrame({"feature": rnd_search.best_estimator_.steps[0][1].get_feature_names_out(), "importance": feature_importances}).sort_values("importance", ascending=False)
important_feature_list


,feature,importance
50,log_pipeline__median_income,0.153449
0,bedrooms_per_room__ratio,0.070693
52,categorical__ocean_proximity_INLAND,0.063040
1,rooms_per_household__ratio,0.047681
2,population_per_household__ratio,0.043118
42,cluster_similarity__similarity_to_cluster_35,0.036243
23,cluster_similarity__similarity_to_cluster_16,0.029481
41,cluster_similarity__similarity_to_cluster_34,0.028111
10,cluster_similarity__similarity_to_cluster_3,0.026634
22,cluster_similarity__similarity_to_cluster_15,0.023397


Interpretation:
* Interesting there are lot of unimportant features, that we can get rid of and may be improve model performance. 
* Lets try and create a list of features to remove and see if we can improve the model
* Lets remove features with importance < 0.05

In [39]:
## features greater than or equal to 0.05
important_feature_list[important_feature_list["importance"] >= 0.05]

,feature,importance
50,log_pipeline__median_income,0.153449
0,bedrooms_per_room__ratio,0.070693
52,categorical__ocean_proximity_INLAND,0.063040


* Interesting we only have 4 features that contributes most to this model.
* This salso indicates that our `hyperparameters` related to geo clusters do not contribute too much to model training. 
* Lets use the complete training dataset, with just these 4 features and see if overfitting is reduced. 
* We will also save the best param model, because for now this is the best model we have and we might end up using it in test/prod. 


#### Saving Model

In [40]:
## Keep this code commented to avoid accidental overwriting of the model
## saving the model to disk
# from joblib import dump

# ## model path
# model_path = Path("..", "models")

# dump(rnd_search.best_estimator_, Path(model_path, "random_forest_model_v0_1.joblib"))

In [41]:
preprocessed_data_reduced = preprocessed_data[important_feature_list[important_feature_list["importance"] >= 0.05]["feature"].values]
preprocessed_data_reduced

,log_pipeline__median_income,bedrooms_per_room__ratio,categorical__ocean_proximity_INLAND
0,-1.078547,1.846624,0.0
1,1.231761,-0.508121,0.0
2,-0.792464,-0.202155,1.0
3,-0.935308,-0.149006,1.0
4,-0.018667,0.963208,0.0
...,...,...,...
16507,0.628497,0.804368,0.0
16508,-0.680615,-0.192328,1.0
16509,0.291670,-0.242492,0.0
16510,0.337576,0.259775,0.0


#### Reduced Features Training

Plan:
* So now we'll train on complete training dataset, with tuned hyper params and reduced feature list
* The hyperparameters will be `max_features = 6` and `n_estimators = 130`
* Assumption is if we get performance similar or slightly better than randomized grid, we are not overfitting the model and we can use it for production. 

In [42]:
## removing preprocessing since we have the preprocessed data
random_forest_pipeline = make_pipeline(RandomForestRegressor(n_estimators=130, max_features=6, random_state=42))

## fit the model
random_forest_pipeline.fit(preprocessed_data_reduced, df_labels)

Pipeline(steps=[('randomforestregressor',
                 RandomForestRegressor(max_features=6, n_estimators=130,
                                       random_state=42))])

In [43]:
## predictions
predictions = random_forest_pipeline.predict(preprocessed_data_reduced)
predictions[:5].round(2)

array([356686.15, 402289.23, 101373.08,  93893.85, 307369.23])

In [44]:
## rmse
rmse = root_mean_squared_error(df_labels, predictions)
rmse

28513.974841034997

* Interesting, the RMSE number is better than Random Search, and feels like less overfitting than before. But we'll have to do cross validation to confirm.

In [45]:
## cross validation
scores = -cross_val_score(random_forest_pipeline, preprocessed_data_reduced, df_labels, scoring="neg_root_mean_squared_error", cv=10)
scores

array([77875.28344894, 76597.1905697 , 74037.87688373, 74493.22378484,
       75220.11749919, 77215.45098697, 76712.29645249, 77721.48812386,
       76750.60993795, 78399.51394145])

* Ok this performs worse than previous cross validation of random forest with mean `rmse` of `~47K`
* So for now, the best model we have is the random search model. Obviously we haven't tested all possible models, and that is because of scope of this project and my limited knowledge in ML. So we'll verify how random search model does on test data and proceed to next part which is creating a simple API.

In [46]:
## testing random search against the test set

## read test data
test_data = pd.read_csv(Path(processed_data_path, "test_set.csv"))

## split the test data into features and labels

test_features = test_data.drop("median_house_value", axis=1)
test_labels = test_data["median_house_value"].copy()

## preprocess the test data
preprocessed_test_data = pd.DataFrame(preprocessing_pipeline.transform(test_features), columns=preprocessing_pipeline.get_feature_names_out())

## predictions using rnd_search
predictions = rnd_search.best_estimator_.predict(test_features)

## rmse
rmse = root_mean_squared_error(test_labels, predictions)

## relative rmse

mean_price = test_labels.mean()
rmse_relative = rmse / mean_price
print(f"RMSE: {rmse}, Relative RMSE: {rmse_relative:.2%}")


RMSE: 41534.12249127612, Relative RMSE: 19.88%


* So we have a model, which has `Relative RMSE 19.85%` which is a lot better than current manual process where the estimates are off by more than 30%.
* The model is approx `33%` better than current manual process.
* We are going to use this as our production model and for the API.

In [56]:
# ## lets create a visualization of predictions vs actual values from test set using plotly
# import plotly.express as px

# ## create a dataframe of predictions and actual values
# predictions_df = pd.DataFrame({"predictions": predictions, "actual": test_labels})

# ## create a scatter plot
# fig = px.scatter(predictions_df, x="actual",y="predictions",  title="Predictions vs Actual Values")
# fig.show()


## lets create a visualization of predictions vs actual values from test set using plotly
import plotly.express as px
import plotly.graph_objects as go

## create a dataframe of predictions and actual values
predictions_df = pd.DataFrame({"predictions": predictions, "actual": test_labels})

## create a scatter plot
fig = px.scatter(predictions_df, x="actual", y="predictions", title="Actual Values vs Predictions", opacity=0.5)

## add a 45-degree reference line
max_value = max(predictions_df["actual"].max(), predictions_df["predictions"].max())
fig.add_trace(
    go.Scatter(x=[0, max_value], y=[0, max_value], mode="lines", name="Ideal Line", line=dict(color="red", dash="solid"))
)

## show the plot
fig.show()

## save the image
fig.write_image(Path("..", "reports", "figures", "final_predictions_vs_actual_values.png"))


Observations:
* Overall it looks like the there is a linear relationship between actual values and predictions, which tells me our model is reasonably accurate in predicting prices.  
* In lower to mid range prices (< 300K), the predictions are closely aligned to actual values. But as the prices go up there are many predictions that go below the ideal line, which tells us that the model is struggling with these predictions. 
* The concentration of points near 500K is due to the upper cap on the prices in data. 

Next Steps:
* We can look into feature engineering to add features correlating to the higher prices. 
* We can try a more complex model.